# simcat workflow
This notebook demonstrates how to use `simcat.Database` objects to simulate a database of invariant matrices representing admixture over all edges of an input topology, and under a range of demographic scenarios and edge lengths. 

In [7]:
import ipcoal
import simcat
import toytree
import numpy as np

In [8]:
# generate a species tree
tree = toytree.rtree.unittree(ntips=6, seed=12345, treeheight=10e6)

In [9]:
tree.draw(ts='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t8a920b2b96944a0b805d21be16e8c526" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r2
dist: 2500000
support: 100
height: 0 0 idx: 1
name: r4
dist: 2500000
support: 100
height: 0 1 idx: 2
name: r3
dist: 5000000
support: 100
height: 0 2 idx: 3
name: r0
dist: 7500000
support: 100
height: 0 3 idx: 4
name: r1
dist: 7500000
support: 100
height: 0 4 idx: 5
name: r5
dist: 7500000
support: 100
height: 0 5 idx: 6
name: 6
dist: 2500000
support: 100
height: 2500000 6 idx: 7
name: 7
dist: 2500000
support: 100
height: 5000000 7 idx: 8
name: 8
dist: 2500000
support: 100
height: 7500000 8 idx: 9
name: 9
dist: 2500000
support: 100
height: 7500000 9 idx: 10
name: 10
dist: 2500000
support: 100
height: 10000000 10 0 5000000 10000000

### generate database with labels

In [16]:
# init a database of simulations
self = simcat.Database(
    name="tr6-t10-r2-s10000", 
    workdir="./databases", 
    tree=tree,
    nedges=1,
    ntests=10,
    nreps=2,
    nsnps=10000,
    force=True,
    Ne=1e5,
)

920 labels to be stored in: ../tests/databases/tr6-t10-r2-s10000.labels.h5


### run ipcoal simulations for each rep

In [ ]:
# set parallelization
self.ipcluster['cores'] = 8
self.run(auto=True)

Box(children=(HTML(value="<span style='font-size:14px; font-family:monospace'>Establishing parallel connection…

Box(children=(HTML(value="<span style='font-size:14px; font-family:monospace'>Parallelization: <i>latituba</i>…

### fit ML models to the data

In [17]:
import simcat.ml as ml

In [ ]:
ml.Analysis()

### visualize features and clustering

### validate (effect of variable node heights)
Now that we have seen the model does very well at predicting admixture edges for data simulated on the same (or similar) tree, let's examine the extent to which *simcat* is correct when the tree is very different. Here we show that node heights have little effect and that by training the model with sliding nodes it fits the data better. 

In [10]:
counts.shape
svds.shape

(860, 70, 16)

In [8]:
import h5py
import pandas as pd

with h5py.File(self.counts) as io5:
    
    # pull in the data
    counts = io5["counts"][:]
    svds = io5["svds"][:]
    
with h5py.File(self.labels) as io5:
    # pull in the labels
    df = pd.DataFrame({
        "asource": io5["admixture"][:, 0].astype(int),
        "atarget": io5["admixture"][:, 1].astype(int),
        "aprop": io5["admixture"][:, 2],
        "atrue": io5["admixture"][:, 2] > 0.05
    })
    
    # categorical label for (source, target) pair
    df["label"] = (
        df.asource.astype(str).str.cat(
            df.atarget.astype(str), sep=","
        )
    )
    
    # non-directional category
    df["ulabel"] = [
        "{},{}".format(*i) for i in df.label.apply(
        lambda x: sorted([int(i) for i in x.split(",")]))
    ]
    
    
df.describe()

,asource,atarget,aprop
count,860.000000,860.000000,860.000000
mean,5.216279,5.190698,0.224543
std,3.488510,3.445353,0.117908
min,0.000000,0.000000,0.079947
25%,2.000000,2.000000,0.103666
50%,5.000000,5.000000,0.240302
75%,8.000000,8.000000,0.289828
max,13.000000,13.000000,0.474675


In [31]:
from sklearn.manifold import TSNE
tsne = TSNE(
    n_components=2,
    perplexity=4.5,
    init='pca',
    n_iter=10000,
    random_state=123,
)

In [46]:
X = counts / counts.max()
X = X.flatten().reshape(-1, 1)

In [ ]:
t = tsne.fit_transform(X)#counts.reshape(860, -1))

In [36]:
color = np.array(['black'] * X.shape[0])
color[]

In [33]:
import toyplot

X = t
toyplot.scatterplot(
    X[:, 0], X[:, 1],
    width=400, height=400,
    title=df.label,
);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="tf47b99be171a496bb4407c4f60a8c525" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2,5 5,1 3,10 4,1 7,5 7,11 0,2 3,8 6,1 3,10 0,2 6,8 2,4 1,5 2,3 4,6 4,0 9,6 7,3 1,9 1,2 4,7 3,8 6,0 9,0 7,8 2,6 11,7 8,7 0,5 0,3 2,11 6,1 6,3 3,9 0,6 7,1 5,1 5,7 8,6 3,1 3,10 4,2 12,13 12,7 9,6 10,11 7,11 4,0 2,8 4,0 1,0 3,1 6,3 7,6 2,3 9,0 3,0 5,4 7,3 7,2 2,8 13,12 0,5 9,7 2,11 4,5 2,11 8,2 9,7 5,7 2,0 3,4 0,2 0,1 1,4 1,0 2,0 12,7 0,2 6,2 5,6 8,6 3,8 3,4 11,12 2,3 8,7 12,11 7,8 6,1 7,10 0,6 8,6 9,2 2,6 12,11 3,0 3,6 6,0 8,9 3,4 3,7 10,6 0,7 5,8 1,5 11,10 0,2 5,4 9,7 10,11 5,0 7,4 12,11 7,2 3,2 4,2 8,3 0,6 8,5 7,2 9,6 5,8 6,2 3,6 7,0 8,6 2,0 4,3 7,5 12,11 8,5 7,9 2,9 8,9 1,3 9,2 5,6 3,4 3,9 2,3 4,0 5,0 7,6 9,10 10,9 2,9 7,0 0,1 5,0 8,2 8,3 0,6 2,0 2,6 5,8 10,6 0,7 5,7 0,6 7,1 0,6 2,9 3,7 3,2 3,4 3,1 7,10 7,1 7,10 8,7 6,8 4,2 1,0 11,7 10,7 3,4 1,6 3,4 6,7 6,8 0,9 10,7 12,13 8,3 13,3 2,3 0,7 2,3 3,0 5,7 13,10 3,9 5,3 1,7 0,5 3,13 9,3 11,8 1,6 1,3 3,5 3,7 4,5 9,8 1,0 7,2 7,4 10,3 9,6 1,7 5,4 5,3 13,3 10,13 1,9 11,8 2,9 4,0 5,3 6,9 9,7 5,3 6,4 13,12 7,11 4,2 1,0 1,7 0,7 1,3 1,4 4,7 5,6 5,4 11,7 0,4 3,11 1,9 8,7 2,6 1,4 6,3 2,8 0,1 2,3 10,7 4,7 3,1 10,3 6,3 9,1 11,10 2,9 1,4 0,3 5,3 7,0 2,5 2,4 7,3 13,12 7,0 3,13 3,5 9,3 6,9 0,5 8,7 13,10 5,6 4,7 5,4 0,3 5,8 6,4 11,10 7,11 4,6 3,4 0,6 1,5 8,6 5,6 2,7 0,5 9,2 8,11 6,2 6,0 5,1 0,3 8,2 6,8 11,3 8,7 4,1 3,1 4,3 11,3 9,6 9,7 3,8 1,0 10,7 3,7 7,1 7,4 6,2 3,0 7,8 8,7 11,2 3,4 8,2 11,2 1,4 13,10 3,8 7,6 6,0 8,5 3,2 7,9 11,10 5,6 1,3 1,4 2,1 8,9 3,9 4,1 5,6 6,7 6,2 1,4 1,2 12,13 13,10 0,3 1,4 1,6 2,3 7,10 8,3 7,3 0,6 6,10 10,13 0,3 4,5 6,10 8,7 2,1 7,11 1,2 9,10 8,2 2,0 1,2 1,5 9,7 0,2 9,7 5,2 5,2 0,5 0,3 13,10 0,4 6,8 10,9 2,6 5,8 10,7 10,11 1,5 6,10 6,0 4,6 8,9 7,2 3,11 5,4 4,7 1,4 2,5 7,0 5,0 11,3 7,0 11,3 5,0 1,2 0,2 7,6 13,3 3,8 6,1 3,5 6,0 2,5 7,0 13,3 1,6 5,3 4,6 9,3 10,6 4,6 6,5 10,9 1,3 9,6 10,13 7,8 9,10 9,8 8,3 8,4 2,6 5,8 3,1 2,4 8,2 1,0 5,3 5,2 2,8 5,4 4,6 11,10 1,5 1,2 7,5 0,2 0,3 11,8 1,3 3,10 8,2 3,8 9,8 0,6 2,1 7,1 7,1 11,3 7,11 7,3 4,1 4,8 11,7 2,9 1,0 11,7 4,7 8,3 1,3 3,4 6,8 6,7 8,7 5,8 0,4 2,3 8,11 4,3 11,3 2,3 2,6 4,2 11,7 9,2 11,10 8,6 2,11 3,4 6,3 3,7 3,1 7,4 1,2 8,9 8,9 4,8 0,7 2,5 1,7 2,4 7,4 7,0 4,0 1,0 0,3 8,6 10,3 8,6 3,11 0,6 3,2 2,0 1,7 11,10 7,6 4,5 3,11 9,8 2,11 10,7 5,1 7,0 8,3 0,3 9,8 4,2 6,2 4,2 8,5 4,6 9,3 4,5 5,4 9,2 2,7 7,10 4,2 3,5 6,2 2,6 12,13 0,1 5,1 5,6 10,9 3,0 0,4 5,4 11,3 3,2 9,3 2,8 1,2 8,6 10,11 4,0 9,2 1,2 7,4 4,8 7,9 7,1 1,7 3,6 9,3 6,5 8,7 3,1 2,5 10,11 8,2 7,11 2,0 4,6 2,1 3,4 10,3 9,3 0,5 0,5 8,9 12,11 3,5 4,5 3,8 4,7 1,0 8,3 5,7 0,4 6,7 8,3 8,3 7,2 8,2 2,9 8,2 12,13 0,4 7,11 5,3 1,6 7,3 4,0 9,3 6,2 9,8 5,0 7,0 4,1 11,7 7,0 3,13 2,7 4,8 6,5 6,7 1,2 3,2 1,4 1,5 6,10 3,4 13,12 5,1 3,13 2,7 7,2 10,3 3,6 11,10 11,10 3,9 0,3 7,11 6,1 3,13 3,1 13,12 1,2 0,4 9,3 5,3 7,9 9,3 0,4 1,5 3,1 4,7 5,6 3,13 11,10 2,0 10,7 4,1 0,1 1,5 3,5 6,0 1,3 3,11 7,5 3,0 12,13 6,3 6,1 2,6 6,8 3,10 7,11 2,6 7,4 8,6 11,3 6,1 4,7 0,3 3,0 4,8 9,8 4,2 7,8 0,6 7,4 3,8 1,5 5,1 11,10 6,7 3,0 6,4 7,11 1,3 6,0 9,10 3,1 5,8 10,3 13,12 12,13 8,2 1,6 0,5 11,3 1,2 3,8 8,6 9,10 10,3 7,8 2,6 3,5 9,8 6,10 3,7 11,7 5,6 5,1 7,8 8,2 9,3 0,4 4,7 4,2 10,13 8,3 2,6 2,6 10,6 5,6 5,0 2,1 5,6 5,3 4,6 6,5 3,10 3,4 2,8 9,7 3,8 4,0 3,13 0,5 3,2 0,4 4,1 1,9 9,2 10,9 12,13 11,3 11,3 1,6 9,8 8,7 6,1 7,5 8,9 9,0 1,4 6,8 3,10 0,2 6,10 0,6 4,1 0,6 3,7 10,11 11,10 2,6 3,10 2,1 10,13 7,8 13,10 0,4 1,6 0,3 6,4 10,13 6,1 4,5 7,10 6,8 1,3 6,1 10,11 4,7 4,5 0,5 8,11 7,2 5,2 0,3 3,6 3,13 10,3 7,10 7,5 9,2 7,2 1,3 9,2 7,1 3,9 2,11 8,6 0,2 3,10 7,6 10,7 3,13 4,7 3,1 6,4 10,

In [22]:
sim = simcat.Simulator.IPCoalWrapper(self.labels, 0, 100, run=False)


In [24]:
sim.load_slice()

In [44]:
idx = 0
tree = sim.tree.mod.node_slider(0.25, seed=sim.slide_seeds[idx])
nes = sim.node_Nes[idx]
admix = (sim.admixture[idx, 0], sim.admixture[idx, 1], 0.5, sim.admixture[idx, 2])
admix
#md = ipcoal.Model(tree, Ne=1e6, admixture_edges=admix)

(0.0, 0.0, 0.5, 0.0)

In [45]:
sim.admixture

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0

In [12]:
md.sim_snps(100)

In [14]:
md.df

,locus,start,end,nbps,nsnps,genealogy
0,0,0,1,1,1,"((r1:7.5444e+06,(r5:5.00..."
1,1,0,1,1,1,"((r1:7.5027e+06,(r5:5.00..."
2,2,0,1,1,1,"((r3:7.506e+06,(r0:5.008..."
3,3,0,1,1,1,"((r1:7.51121e+06,(r5:5.0..."
4,4,0,1,1,1,"((r3:7.50542e+06,(r2:5.0..."
...,...,...,...,...,...,...
95,95,0,1,1,1,"((r3:7.51874e+06,(r2:5.0..."
96,96,0,1,1,1,"((r3:7.51738e+06,(r0:5.1..."
97,97,0,1,1,1,"((r3:7.5094e+06,(r2:5.02..."
98,98,0,1,1,1,"((r3:7.50704e+06,(r0:5.0..."


In [32]:
from simcat.Simulator import IPCoalWrapper
sim = IPCoalWrapper(self.labels, 0, 10, run=True)

In [8]:
sim.run()

In [34]:
sim.vector.shape

(10, 4176)

In [47]:
(sim.nquarts * 16 * 16 ) * 3

3840

In [50]:
(sim.nquarts * 16)

80

In [53]:
3840 + 80 + (16 * 16)

4176

In [13]:
sim.vector.shape

(10, 55136)

In [25]:
55136 / (sim.nquarts * 16 * 16)

3.0767857142857142

In [19]:
vectorsnps.shape

(10, 17920)

In [23]:
16 * 16 * sim.nquarts

17920

In [17]:
u, s, vh = np.linalg.svd(sim.counts)

# compute variance (10, 15, 16, 16) -> (10, 16, 16)
mvar = sim.counts.var(axis=1)

# reshape to ntests flattened (10, 15, 16, 16) -> (10, 3840)
vectorsnps = sim.counts.reshape(sim.counts.shape[0], -1)

In [16]:
vectorsnps.shape

(10, 17920)

In [25]:
np.concatenate([
            vectorsnps,
            u.reshape(u.shape[0], -1),
            s.reshape(s.shape[0], -1),
            vh.reshape(vh.shape[0], -1),
            mvar.reshape(mvar.shape[0], -1),
        ], axis=1).shape

(10, 55136)

In [23]:
np.concatenate([vectorsnps, u.reshape(u.shape[0], -1)], axis=1).shape

(10, 35840)

In [44]:
u, s, vh = np.linalg.svd(sim.counts[0, 0])
s

array([104.63945579,  93.39556673,  90.88989671,  79.61523804,
        12.37101979,   5.82287683,   5.70067032,   4.93675316,
         3.9519926 ,   3.3939291 ,   2.49925724,   2.28251045,
         1.69366961,   1.44245668,   0.95794008,   0.12556873])

In [88]:
u, s, vh = np.linalg.svd(sim.counts, full_matrices=True)
s.shape

(10, 70, 16)

In [71]:
np.concatenate([u.flatten(), s.flatten(), vh.flatten()]).shape
sim.counts.ravel()

array([85, 12,  9, ..., 17, 15, 86])

In [ ]:
>>> arr = numpy.zeros((50,100,25))
>>> new_arr = arr.reshape(-1, arr.shape[-1])
>>> new_arr.shape
# (5000, 25)

In [ ]:
sim.counts.

In [84]:
15 * 16 * 16

3840

In [82]:
sim.counts.var(axis=1).shape

(10, 16, 16)

In [83]:
sim.counts.reshape(10, -1).shape

(10, 17920)

In [72]:
sim.counts.shape

(10, 70, 16, 16)

In [51]:
np.concatenate([s, sim.counts], axis=1)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 4 dimension(s)

In [40]:
s[0, 0]

array([104.63945579,  93.39556673,  90.88989671,  79.61523804,
        12.37101979,   5.82287683,   5.70067032,   4.93675316,
         3.9519926 ,   3.3939291 ,   2.49925724,   2.28251045,
         1.69366961,   1.44245668,   0.95794008,   0.12556873])

In [14]:
import simcat.plot

In [19]:
simcat.plot.draw_count_matrix(sim.counts[0, 0]);

<svg class="toyplot-canvas-Canvas" height="750.0px" id="t8dcb0890ff3844728817078451c59aff" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 750.0 750.0" width="750.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.000000 AAAA 0.625000 AAAC 0.552885 AAAG 0.514423 AAAT 0.389423 AACA 0.028846 AACC 0.014423 AACG 0.014423 AACT 0.360577 AAGA 0.019231 AAGC 0.028846 AAGG 0.019231 AAGT 0.408654 AATA 0.000000 AATC 0.014423 AATG 0.052885 AATT 1 0.725962 ACAA 0.173077 ACAC 0.048077 ACAG 0.038462 ACAT 0.033654 ACCA 0.370192 ACCC 0.014423 ACCG 0.014423 ACCT 0.009615 ACGA 0.028846 ACGC 0.014423 ACGG 0.004808 ACGT 0.028846 ACTA 0.009615 ACTC 0.000000 ACTG 0.019231 ACTT 2 1.000000 AGAA 0.048077 AGAC 0.187500 AGAG 0.062500 AGAT 0.009615 AGCA 0.043269 AGCC 0.024038 AGCG 0.004808 AGCT 0.048077 AGGA 0.024038 AGGC 0.350962 AGGG 0.009615 AGGT 0.028846 AGTA 0.004808 AGTC 0.014423 AGTG 0.024038 AGTT 3 0.841346 ATAA 0.052885 ATAC 0.028846 ATAG 0.201923 ATAT 0.038462 ATCA 0.019231 ATCC 0.000000 ATCG 0.019231 ATCT 0.014423 ATGA 0.004808 ATGC 0.028846 ATGG 0.000000 ATGT 0.043269 ATTA 0.009615 ATTC 0.004808 ATTG 0.403846 ATTT 4 0.365385 CAAA 0.052885 CAAC 0.024038 CAAG 0.009615 CAAT 0.182692 CACA 0.875000 CACC 0.048077 CACG 0.014423 CACT 0.009615 CAGA 0.024038 CAGC 0.038462 CAGG 0.000000 CAGT 0.024038 CATA 0.014423 CATC 0.009615 CATG 0.009615 CATT 5 0.038462 CCAA 0.509615 CCAC 0.004808 CCAG 0.014423 CCAT 0.581731 CCCA 0.000000 CCCC 0.596154 CCCG 0.524038 CCCT 0.004808 CCGA 0.336538 CCGC 0.038462 CCGG 0.019231 CCGT 0.014423 CCTA 0.379808 CCTC 0.009615 CCTG 0.043269 CCTT 6 0.028846 CGAA 0.014423 CGAC 0.000000 CGAG 0.004808 CGAT 0.067308 CGCA 0.889423 CGCC 0.230769 CGCG 0.038462 CGCT 0.009615 CGGA 0.062500 CGGC 0.394231 CGGG 0.004808 CGGT 0.004808 CGTA 0.033654 CGTC 0.024038 CGTG 0.019231 CGTT 7 0.014423 CTAA 0.014423 CTAC 0.000000 CTAG 0.014423 CTAT 0.019231 CTCA 0.846154 CTCC 0.048077 CTCG 0.216346 CTCT 0.000000 CTGA 0.014423 CTGC 0.004808 CTGG 0.009615 CTGT 0.000000 CTTA 0.086538 CTTC 0.000000 CTTG 0.298077 CTTT 8 0.432692 GAAA 0.004808 GAAC 0.038462 GAAG 0.019231 GAAT 0.004808 GACA 0.019231 GACC 0.004808 GACG 0.000000 GACT 0.201923 GAGA 0.038462 GAGC 0.750000 GAGG 0.028846 GAGT 0.024038 GATA 0.000000 GATC 0.024038 GATG 0.014423 GATT 9 0.009615 GCAA 0.009615 GCAC 0.024038 GCAG 0.004808 GCAT 0.019231 GCCA 0.427885 GCCC 0.052885 GCCG 0.014423 GCCT 0.024038 GCGA 0.149038 GCGC 0.855769 GCGG 0.028846 GCGT 0.000000 GCTA 0.019231 GCTC 0.033654 GCTG 0.019231 GCTT 10 0.033654 GGAA 0.019231 GGAC 0.427885 GGAG 0.000000 GGAT 0.019231 GGCA 0.033654 GGCC 0.423077 GGCG 0.009615 GGCT 0.576923 GGGA 0.596154 GGGC 0.000000 GGGG 0.610577 GGGT 0.014423 GGTA 0.019231 GGTC 0.375000 GGTG 0.033654 GGTT 11 0.014423 GTAA 0.000000 GTAC 0.019231 GTAG 0.019231 GTAT 0.004808 GTCA 0.028846 GTCC 0.019231 GTCG 0.014423 GTCT 0.024038 GTGA 0.033654 GTGC 0.826923 GTGG 0.288462 GTGT 0.009615 GTTA 0.028846 GTTC 0.038462 GTTG 0.312500 GTTT 12 0.394231 TAAA 0.033654 TAAC 0.014423 TAAG 0.033654 TAAT 0.033654 TACA 0.009615 TACC 0.009615 TACG 0.028846 TACT 0.014423 TAGA 0.000000 TAGC 0.024038 TAGG 0.038462 TAGT 0.216346 TATA 0.038462 TATC 0.052885 TATG 0.889423 TATT 13 0.019231 TCAA 0.033654 TCAC 0.000000 TCAG 0.024038 TCAT 0.024038 TCCA 0.423077 TCCC 0.004808 TCCG 0.033654 TCCT 0.004808 TCGA 0.024038 TCGC 0.009615 TCGG 0.009615 TCGT 0.019231 TCTA 0.221154 TCTC 0.033654 TCTG 0.918269 TCTT 14 0.028846 TGAA 0.004808 TGAC 0.014423 TGAG 0.009615 TGAT 0.004808 TGCA 0.004808 TGCC 0.014423 TGCG 0.033654 TGCT 0.009615 TGGA 0.014423 TGGC 0.423077 TGGG 0.043269 TGGT 0.024038 TGTA 0.028846 TGTC 0.216346 TGTG 0.884615 TGTT 15 0.024038 TTAA 0.0

In [4]:
ipcoal.Model(tree, Ne=10000, admixture_edges=(5, 3, (0.25, 0.75), 0.5), debug=True)

div time:    3000000,  4  0, 13 12, Ne=10000
div time:    2250000,  7  4,  7 11, Ne=10000
div time:    2250000,  3  0,  3 10, Ne=10000
div time:    1500000,  6  4,  6  9, Ne=10000
div time:    1500000,  2  0,  2  8, Ne=10000
div time:     750000,  5  4,  5  4, Ne=10000
div time:     750000,  1  0,  1  0, Ne=10000
mig pulse:    348925,      ,  5  3, rate=0.50
pop: Ne:10000, mut:1.00E-08


In [76]:
aedges = simcat.utils.get_all_admix_edges(tree, 0.5, 0.5, True)
#aes = np.random.choice(range(len(aedges)), 12)
#np.array(list(aedges))[aes]
aedges

{(7, 3): (1125000.0, 1125000.0),
 (7, 10): (1875000.0, 1875000.0),
 (7, 6): (750000.0, 750000.0),
 (7, 9): (1125000.0, 1125000.0),
 (7, 2): (750000.0, 750000.0),
 (7, 8): (1125000.0, 1125000.0),
 (7, 5): (375000.0, 375000.0),
 (7, 4): (375000.0, 375000.0),
 (7, 1): (375000.0, 375000.0),
 (7, 0): (375000.0, 375000.0),
 (11, 3): (1875000.0, 1875000.0),
 (11, 10): (1875000.0, 1875000.0),
 (3, 7): (1125000.0, 1125000.0),
 (3, 11): (1875000.0, 1875000.0),
 (3, 6): (750000.0, 750000.0),
 (3, 9): (1125000.0, 1125000.0),
 (3, 2): (750000.0, 750000.0),
 (3, 8): (1125000.0, 1125000.0),
 (3, 5): (375000.0, 375000.0),
 (3, 4): (375000.0, 375000.0),
 (3, 1): (375000.0, 375000.0),
 (3, 0): (375000.0, 375000.0),
 (10, 7): (1875000.0, 1875000.0),
 (10, 11): (1875000.0, 1875000.0),
 (6, 7): (750000.0, 750000.0),
 (6, 3): (750000.0, 750000.0),
 (6, 2): (750000.0, 750000.0),
 (6, 8): (1125000.0, 1125000.0),
 (6, 5): (375000.0, 375000.0),
 (6, 4): (375000.0, 375000.0),
 (6, 1): (375000.0, 375000.0),
 (6, 

In [90]:
tree.draw(ts='c');

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tdfafd0a1be624b9fbb7461db14f0c71d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 250.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> idx: 0
name: r2
dist: 2500000
support: 100
height: 0 0 idx: 1
name: r7
dist: 2500000
support: 100
height: 0 1 idx: 2
name: r0
dist: 5000000
support: 100
height: 0 2 idx: 3
name: r3
dist: 7500000
support: 100
height: 0 3 idx: 4
name: r6
dist: 2500000
support: 100
height: 0 4 idx: 5
name: r4
dist: 2500000
support: 100
height: 0 5 idx: 6
name: r5
dist: 5000000
support: 100
height: 0 6 idx: 7
name: r1
dist: 7500000
support: 100
height: 0 7 idx: 8
name: 8
dist: 2500000
support: 100
height: 2500000 8 idx: 9
name: 9
dist: 2500000
support: 100
height: 2500000 9 idx: 10
name: 10
dist: 2500000
support: 100
height: 5000000 10 idx: 11
name: 11
dist: 2500000
support: 100
height: 5000000 11 idx: 12
name: 12
dist: 2500000
support: 100
height: 7500000 12 idx: 13
name: 13
dist: 2500000
support: 100
height: 7500000 13 idx: 14
name: 14
dist: 2500000
support: 100
height: 10000000 14 0 5000000 10000000

In [15]:
# iter over edges
for edge in db.aedges:
    
    # iter over params
    for test in range(db.ntests):
        
        Ne = np.random.uniform(10000, 100000)
        
        # replicate
        for rep in range(db.nreps):
            print(edge, test)
        
            model = ipcoal.Model(
                tree=tree,
                Ne=Ne,
#                 admixture_edges=[()]
            )

((4, 7), (4, 3)) 0


AttributeError: 'Model' object has no attribute '_get_migration'

In [3]:
import ipcoal
import itertools

In [5]:
# for i in range(700):
#     model = ipcoal.Model(
#         tree=tree,
#         Ne=1000000,
#         admixture_edges=(1, 2, (0.2, 0.5), (0.1, 0.9)),
#         admixture_type=0,
#     )
#     model.test_values

In [6]:
# model.sim_snps(100)

### SIMCAT DATABASE

1. Take *range* of parameters for X simulations and generate h5 database of input labels that will used for all X simulations.


2. Distribute parallel jobs to call ipcoal.Model(...).sim_snps(nsnps) on each set of labels.


In [7]:
#simcat.utils.get_all_admix_edges()

In [8]:
# init a database of simulations
db = simcat.Database(
    name="tr5-t10-r10-s10000", 
    workdir="./databases", 
    tree=tree, 
    nedges=1,
    ntests=10,
    nreps=2,
    nsnps=10000,
    force=True,
)

640 labels to be stored in: ../tests/databases/tr5-t10-r10-s10000.labels.h5


In [17]:
#ipcoal.Model(tree, samples=2).sim_snps(1)

### View resulting databases

In [59]:
# the counts array (matrix of sim 0, quartet 0)
with h5py.File(db.counts) as io5:
    counts = io5['counts'][0, 0]
    simcat.plot.draw_count_matrix(
        counts, 
        show_invariants=False, 
        height="600px", 
        width="600px",
        **{"font-size": "8px"}
    )

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t140b2ba2c81a4b22bef4ddda787c731d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 600.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0 0.000000 AAAA 0.900000 AAAC 0.766667 AAAG 0.600000 AAAT 0.433333 AACA 0.233333 AACC 0.000000 AACG 0.000000 AACT 0.200000 AAGA 0.000000 AAGC 0.233333 AAGG 0.000000 AAGT 0.400000 AATA 0.000000 AATC 0.000000 AATG 0.133333 AATT 1 0.366667 ACAA 0.066667 ACAC 0.000000 ACAG 0.000000 ACAT 0.000000 ACCA 0.200000 ACCC 0.000000 ACCG 0.000000 ACCT 0.000000 ACGA 0.000000 ACGC 0.000000 ACGG 0.000000 ACGT 0.000000 ACTA 0.000000 ACTC 0.000000 ACTG 0.000000 ACTT 2 0.166667 AGAA 0.000000 AGAC 0.033333 AGAG 0.000000 AGAT 0.000000 AGCA 0.000000 AGCC 0.000000 AGCG 0.000000 AGCT 0.033333 AGGA 0.000000 AGGC 0.466667 AGGG 0.000000 AGGT 0.000000 AGTA 0.000000 AGTC 0.000000 AGTG 0.000000 AGTT 3 0.200000 ATAA 0.000000 ATAC 0.000000 ATAG 0.000000 ATAT 0.000000 ATCA 0.000000 ATCC 0.000000 ATCG 0.000000 ATCT 0.000000 ATGA 0.000000 ATGC 0.000000 ATGG 0.000000 ATGT 0.000000 ATTA 0.000000 ATTC 0.000000 ATTG 0.200000 ATTT 4 0.300000 CAAA 0.066667 CAAC 0.000000 CAAG 0.000000 CAAT 0.000000 CACA 0.200000 CACC 0.000000 CACG 0.000000 CACT 0.000000 CAGA 0.000000 CAGC 0.000000 CAGG 0.000000 CAGT 0.000000 CATA 0.000000 CATC 0.000000 CATG 0.000000 CATT 5 0.066667 CCAA 0.500000 CCAC 0.000000 CCAG 0.000000 CCAT 0.533333 CCCA 0.000000 CCCC 0.833333 CCCG 1.000000 CCCT 0.000000 CCGA 0.566667 CCGC 0.100000 CCGG 0.000000 CCGT 0.000000 CCTA 0.366667 CCTC 0.000000 CCTG 0.133333 CCTT 6 0.000000 CGAA 0.000000 CGAC 0.000000 CGAG 0.000000 CGAT 0.000000 CGCA 0.266667 CGCC 0.033333 CGCG 0.000000 CGCT 0.000000 CGGA 0.066667 CGGC 0.366667 CGGG 0.000000 CGGT 0.000000 CGTA 0.000000 CGTC 0.000000 CGTG 0.000000 CGTT 7 0.000000 CTAA 0.000000 CTAC 0.000000 CTAG 0.000000 CTAT 0.000000 CTCA 0.233333 CTCC 0.000000 CTCG 0.033333 CTCT 0.000000 CTGA 0.000000 CTGC 0.000000 CTGG 0.000000 CTGT 0.000000 CTTA 0.066667 CTTC 0.000000 CTTG 0.400000 CTTT 8 0.333333 GAAA 0.000000 GAAC 0.033333 GAAG 0.000000 GAAT 0.000000 GACA 0.000000 GACC 0.000000 GACG 0.000000 GACT 0.000000 GAGA 0.000000 GAGC 0.300000 GAGG 0.000000 GAGT 0.000000 GATA 0.000000 GATC 0.000000 GATG 0.000000 GATT 9 0.000000 GCAA 0.000000 GCAC 0.000000 GCAG 0.000000 GCAT 0.000000 GCCA 0.400000 GCCC 0.000000 GCCG 0.000000 GCCT 0.000000 GCGA 0.033333 GCGC 0.366667 GCGG 0.000000 GCGT 0.000000 GCTA 0.000000 GCTC 0.000000 GCTG 0.000000 GCTT 10 0.200000 GGAA 0.000000 GGAC 0.600000 GGAG 0.000000 GGAT 0.000000 GGCA 0.066667 GGCC 0.400000 GGCG 0.000000 GGCT 0.533333 GGGA 0.566667 GGGC 0.000000 GGGG 0.900000 GGGT 0.000000 GGTA 0.000000 GGTC 0.266667 GGTG 0.300000 GGTT 11 0.000000 GTAA 0.000000 GTAC 0.000000 GTAG 0.000000 GTAT 0.000000 GTCA 0.000000 GTCC 0.000000 GTCG 0.000000 GTCT 0.000000 GTGA 0.000000 GTGC 0.266667 GTGG 0.033333 GTGT 0.000000 GTTA 0.000000 GTTC 0.033333 GTTG 0.400000 GTTT 12 0.300000 TAAA 0.000000 TAAC 0.000000 TAAG 0.000000 TAAT 0.000000 TACA 0.000000 TACC 0.000000 TACG 0.000000 TACT 0.000000 TAGA 0.000000 TAGC 0.000000 TAGG 0.000000 TAGT 0.033333 TATA 0.000000 TATC 0.000000 TATG 0.300000 TATT 13 0.000000 TCAA 0.000000 TCAC 0.000000 TCAG 0.000000 TCAT 0.000000 TCCA 0.466667 TCCC 0.000000 TCCG 0.000000 TCCT 0.000000 TCGA 0.000000 TCGC 0.000000 TCGG 0.000000 TCGT 0.000000 TCTA 0.033333 TCTC 0.000000 TCTG 0.266667 TCTT 14 0.000000 TGAA 0.000000 TGAC 0.000000 TGAG 0.000000 TGAT 0.000000 TGCA 0.000000 TGCC 0.000000 TGCG 0.000000 TGCT 0.000000 TGGA 0.000000 TGGC 0.433333 TGGG 0.000000 TGGT 0.000000 TGTA 0.000000 TGTC 0.000000 TGTG 0.333333 TGTT 15 0.100000 TTAA 0.0

In [53]:
# the counts array (matrix of sim 0, quartet 0)
with h5py.File(db.labels) as io5:
    print(io5["thetas"][0])
    print(io5["admix_sources"][0])
    print(io5["admix_targets"][0])
    print(io5["admix_times"][0])
    print(io5["admix_props"][0])
    print(io5.attrs['tree'])
    print(io5.attrs['nsnps'])

0.39901473699216816
[4]
[7]
[2.625]
[0.49896081]
(4:3,(3:2.25,(2:1.5,(1:0.75,0:0.75)1:0.75)1:0.75)1:0.75);
1000
